In [2]:
%load_ext pycodestyle_magic
%flake8_on

import pandas as pd
import math
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm
import random
import time
import multiprocessing as mp
import numpy as np
import random
import time
import multiprocessing as mp
import numpy as np

---

In [8]:
voids = pd.read_csv('data/voids.csv', header=None)
voids.columns = ['Sample', 'ID', 'RAdeg', 'DEdeg', 'z',
                 'NGal', 'V', 'Reff', 'nmin', 'delmin', 'r',
                 'P', 'Dbound']

In [9]:
voids

,Sample,ID,RAdeg,DEdeg,z,NGal,V,Reff,nmin,delmin,r,P,Dbound
0,CMASS North,60,114.782,37.641,0.648,35,141000.0,32.298,0.000025,-0.717,3.922,3.220000e-14,52.504
1,CMASS North,10020,184.261,1.326,0.500,25,17000.0,15.964,0.000136,-0.652,3.441,2.200000e-10,28.489
2,CMASS North,11496,124.855,3.090,0.648,117,605000.0,52.473,0.000019,-0.778,3.372,6.630000e-10,54.891
3,CMASS North,15935,230.976,13.239,0.459,83,243000.0,38.683,0.000031,-0.876,3.328,1.330000e-09,57.265
4,CMASS North,4407,237.406,16.985,0.463,372,1070000.0,63.467,0.000029,-0.884,3.001,1.330000e-07,73.644
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,LOWZ South,1297,344.903,0.155,0.326,252,429000.0,46.798,0.000060,-0.816,1.584,4.220000e-02,52.979
1224,LOWZ South,708,323.714,-0.573,0.343,55,106000.0,29.356,0.000074,-0.749,1.581,4.290000e-02,48.575
1225,LOWZ South,69,322.872,4.201,0.347,196,634000.0,53.279,0.000051,-0.807,1.579,4.340000e-02,78.259
1226,LOWZ South,1181,12.548,5.708,0.377,331,864000.0,59.086,0.000055,-0.774,1.577,4.380000e-02,86.464


In [6]:
ard_not_adv = pd.read_csv('data/ard_not_adv.csv', skiprows=0)

In [7]:
ard_not_adv

,Unnamed: 0,Unnamed: 0.1,amf_no,amf_ra,amf_dec,amf_z,amf_lk,amf_rh,amf_r200,amf_rc,...,red_id,whl_id,glat,glon,amf_sn,intensity100,intensity857,intensity143,intensity353,cluster
0,0,0,5.0,260.6324,32.1398,0.2252,222.1219,200.7117,1.784,0.625,...,RMJ172227.2+320757.2,J172227.2+320757,31.846666,55.602221,24.655148,-0.000048,1.978746,-0.000006,0.001062,1
1,1,1,7.0,250.1485,46.6917,0.2248,225.9874,182.3373,1.725,0.495,...,RMJ164019.8+464241.5,J164019.8+464242,41.429880,72.599428,44.674799,-0.000269,1.196847,-0.000215,0.000722,1
2,3,3,10.0,203.1873,50.5346,0.3110,164.8704,179.1883,1.663,0.422,...,RMJ133238.4+503336.0,J133252.1+503134,65.305106,107.103759,25.696075,-0.000124,1.019871,-0.000129,0.000580,1
3,4,4,12.0,350.2432,29.2029,0.5231,165.0015,174.7108,1.518,0.390,...,RMJ232104.1+291134.5,J232104.1+291134,-29.646780,100.211077,13.793623,0.000047,4.913466,0.000091,0.001929,1
4,5,5,13.0,139.4938,51.7236,0.2574,169.5809,172.6889,1.676,0.199,...,RMJ091753.4+514337.5,J091753.4+514338,43.387615,166.090009,20.839521,0.000029,1.074994,0.000060,0.000894,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21001,21533,21533,54265.0,117.7965,28.4314,0.4498,21.3133,20.0019,0.721,0.090,...,-9999,J075110.0+282537,24.760679,192.272829,0.235375,-0.000085,1.742793,-0.000074,0.000728,1
21002,21534,21534,54266.0,10.2234,16.4729,0.3325,7.5789,20.0018,0.754,0.420,...,-9999,J004043.6+162754,-46.323866,119.270217,1.128670,-0.000071,1.821485,-0.000079,0.000611,1
21003,21535,21535,54272.0,332.5373,16.8053,0.2720,19.0528,20.0008,0.774,0.059,...,-9999,J221009.3+165449,-31.180104,76.542726,0.327218,-0.000112,2.181500,-0.000118,0.000789,1
21004,21536,21536,54273.0,321.0614,-5.5885,0.4036,7.3319,20.0008,0.734,0.260,...,-9999,J212412.8-053647,-36.312474,46.861993,0.906488,0.000283,3.162386,0.000297,0.001527,1


---

In [4]:
pl = hp.read_map("data/HFI_SkyMap_143-field-IQU_2048_R3.00_full.fits")
pl[pl < -1e23] = 0

/home/maamari/Documents/Other/Anaconda/lib/python3.7/site-packages/healpy/fitsfunc.py:352: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  "If you are not specifying the input dtype and using the default "
/home/maamari/Documents/Other/Anaconda/lib/python3.7/site-packages/healpy/fitsfunc.py:403: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  warnings.warn("No INDXSCHM keyword in header file : " "assume {}".format(schm))


NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING


In [5]:
shape, wcs = enmap.fullsky_geometry(res=5.0*utils.arcmin, proj='car')

In [6]:
map_pix = reproject.enmap_from_healpix(pl, shape=shape, wcs=wcs)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


---

In [7]:
signal = []
ras_temp = []
decs_temp = []

ras_fake = voids['RAdeg']
decs_fake = voids['DEdeg']

N = len(ras_fake)
# N=10
stack_CMB_kSZ           = 0
stack_CMB_kSZ_deproject = 0
c = 0

10:14: E221 multiple spaces before operator


In [8]:
# def worker(i):
#     for j in range(N):
#         print(j/N)    
#         stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs_fake[i], ras_fake[j]]),r=3*utils.arcmin)

#         if stamp is None:
#             continue
#         elif stamp[0][0][0]==0.0:
#             continue

#         signal.append(sum(sum(stamp[0]))/9)
#         ras_temp.append(ras_fake[j])
#         decs_temp.append(decs_fake[i])
        
#     return [signal, ras_temp, decs_temp]
        
# # pool = mp.Pool(processes=8)
# # for _ in tqdm(pool.imap_unordered(worker, range(N)), total=len(range(N))):
# #     pass

# with mp.Pool(processes = 1) as p:
#     results = p.map(worker, [x for x in range(N)])

3:21: W291 trailing whitespace
4:80: E501 line too long (113 > 79 characters)
14:1: W293 blank line contains whitespace
16:1: W293 blank line contains whitespace


In [9]:
signal = []
ras_temp = []
decs_temp = []

for i in tqdm(range(N)):
    stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs_fake[i], ras_fake[i]]),r=3*utils.arcmin)

    if stamp is None:
        continue
    elif stamp[0][0][0]==0.0:
        continue

    signal.append(sum(sum(stamp[0]))/9)
    ras_temp.append(ras_fake[i])
    decs_temp.append(decs_fake[i])

100%|██████████| 1228/1228 [00:11<00:00, 108.25it/s]
6:41: E231 missing whitespace after ','
6:48: E251 unexpected spaces around keyword / parameter equals
6:50: E251 unexpected spaces around keyword / parameter equals
6:80: E501 line too long (107 > 79 characters)
6:90: E231 missing whitespace after ','
10:24: E225 missing whitespace around operator


In [10]:
np.savetxt("voids_signal143.csv",
           signal,
           delimiter=", ",
           fmt="% s")

In [ ]:
np.savetxt("results353_1.csv", 
           results[:10][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_2.csv", 
           results[10:20][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_3.csv", 
           results[20:30][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_4.csv", 
           results[30:40][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_5.csv", 
           results[40:50][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_6.csv", 
           results[50:60][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_7.csv", 
           results[60:70][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_8.csv", 
           results[70:80][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_9.csv", 
           results[80:90][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_10.csv", 
           results[90:100][0],
           delimiter =", ", 
           fmt ='% s')